In [6]:
# Matplotlib
%matplotlib inline
import matplotlib as mpl
from matplotlib import pylab
import matplotlib.pyplot as plt
pylab.rcParams['figure.figsize'] = (10.0, 7.0)
mpl.style.use('ggplot')
from matplotlib.backends.backend_pgf import FigureCanvasPgf
mpl.backend_bases.register_backend('pgf', FigureCanvasPgf)

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

import sys, os
sys.path.append("../tools/")
from collections import defaultdict

# Data management libraries
import pandas as pd
#import seaborn as sns

# Treebank utils, found in "../tools"
import udeval, lang_utils, dependency_classes, udtree

is_labeled = True
is_fine_grained_deprels = False
file_format = "conllx"
test_type = "test"

system_output_base = "/Users/jimmy/dev/edu/nlp-rod/udeval/resources/maltdefault_coarse_output_test_1-2/"
gold_output_base = "/Users/jimmy/dev/edu/nlp-rod/udeval/resources/universaldependencies1-2/universal-dependencies-1.2/"
file_prefix = "maltdefault"

system_files = lang_utils.get_system_output_paths(system_output_base, type_=test_type, format_=file_format)
gold_files = lang_utils.get_ud_paths(gold_output_base, type_=test_type, format_=file_format, coarse=not is_fine_grained_deprels)

def labeled_attachment_scores():
    las = {}
    for lang, system_path in system_files.items():
        s_out = system_path
        g_out = gold_files[lang]
        score = udeval.attachment_score(s_out, g_out, labeled=is_labeled, fine_grained_deprels=is_fine_grained_deprels)
        las[lang] = score
    return pd.Series(las)

def averaged_weighted_attachment_scores():
    wlas = {}
    wde = pd.Series.from_csv('../data/function_content_degree.wde.csv').to_dict()
    wde['punct'] = 0
    for lang, system_path in system_files.items():
        s_out = system_path
        g_out = gold_files[lang]
        score = udeval.weighted_las(s_out, g_out, wde)
        wlas[lang] = score
    return pd.Series(wlas)

In [14]:
gtrees = list(udtree.from_files(gold_files['Swedish']))
strees = list(udtree.from_files(system_files['Swedish']))

In [23]:
wde = pd.Series.from_csv('../data/function_content_degree.wde.csv').to_dict()
wde['punct'] = 0
tot = sum([wde[x] for x in gtrees[1].deprels])

In [24]:
print([x for x in strees[1].deprels])
sum([wde[x] for x in gtrees[1].deprels][1:]) / tot

['nsubj', 'root', 'nsubj', 'case', 'mwe', 'det', 'dobj', 'punct']


0.80880515194556057

In [44]:
udeval.root_distance_las('../resources/universaldependencies1-2/universal-dependencies-1.2/UD_Swedish/sv-ud-test.conllx', '../resources/maltdefault_coarse_output_test_1-2/Swedish.conllx')

0.7788445254641856